<a href="https://colab.research.google.com/github/yukinaga/bert_nlp/blob/main/section_5/01_news_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 学習データ、テストデータの整頓
## data_cleaning.ipynb
データを整理整頓するプログラムです。

### 分類精度検証用の科研費データ読み込み

In [4]:
import pandas as pd

print("Start")

# 読み込むcsvファイル名
# open_original_csv = "RUC共同研究_概要_NoJSPS.csv" #
# open_original_csv = "RUC共同研究_概要_NoDescription.csv" # 
# open_original_csv = "再々依頼_概要なし.csv" # 
open_original_csv = "再々依頼_概要あり.csv" # 

# Column名
ColName_abst = "description"
ColName_true = "" # if no ground truth, then blank.


data_path = "../../data/"

# csvファイルを開く
df = pd.read_csv(data_path + open_original_csv) # dtype="object"必要？




# 読み込んだデータをチェック
df.info()
df

Start
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2702 entries, 0 to 2701
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              2702 non-null   object 
 1   FA              2702 non-null   object 
 2   Title           2702 non-null   object 
 3   description     2702 non-null   object 
 4   review_section  179 non-null    object 
 5   大区分             0 non-null      float64
 6   value           0 non-null      float64
dtypes: float64(2), object(5)
memory usage: 147.9+ KB


,ID,FA,Title,description,review_section,大区分,value
0,1009959,環境省,短寿命α線核種の合理的規制のためのデータ取得による安全性検証と安全管理・教育方法の開発,"・211At,223Ra および 225Ac と壊変核種について、精製・標識実験、細胞実験、...",NaN,NaN,NaN
1,918584,AMED,モデル動物等を用いたHCV感染病態と関連する宿主・ウイルス因子の解析と新規治療法の開発に関する研究,・ヒト肝細胞キメラ TK-NOG マウスに C 型肝炎患者血清を投与することにより、ウイルス...,NaN,NaN,NaN
2,886054,厚労省,医療機関外死亡における死後画像診断の実施に関する研究,・医療機関外死亡に対する死後画像診断を安全に実施するための基準を作成する。・死後画像診断につ...,NaN,NaN,NaN
3,955716,厚労省,新興・再興感染症の発生に備えた感染症サーベイランスの強化とリスクアセスメント,・感染症発生動向調査の評価と改善法の提案：疫学的・統計学的な観点からの評価とともに、ステーク...,NaN,NaN,NaN
4,918227,AMED,進行頭頸部がんに対する術後補助療法の標準治療確立のための多施設共同研究,・局所進行頭頸部扁平上皮癌術後再発High-Risk患者に対するweekly CDDP+RT...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2697,918254,AMED,人工核酸YB－1阻害アンチセンス：膵癌に対する新しい分子標的治療の開発,膵癌に対する高い抗腫瘍効果が認められた人工核酸YB-1阻害アンチセンスを臨床応用するために、...,NaN,NaN,NaN
2698,917996,AMED,RNA測定による膵癌血液診断法の実用化研究,膵癌は、我が国で約3万人が年間死亡し、部位別癌死亡数の第5位を占める重要な悪性腫瘍である。膵...,NaN,NaN,NaN
2699,918255,AMED,膵癌症例の術後転移再発抑制を目差した慢性肝炎治療薬3－オキシゲルミルプロピオン酸重合体を用い...,膵癌は浸潤・転移能が非常に高く、転移抑制剤は膵癌の進行を抑制する戦略として期待される。われわ...,NaN,NaN,NaN
2700,147197,厚労省,希少疾患への治療応用を目指した臍帯および臍帯血由来細胞の系統的資源化とその応用に関する研究,臍帯血は血液疾患のみならず、組織幹細胞ソースとして代謝性疾患等にも応用され、さらに近年では脳...,NaN,NaN,NaN


In [ ]:
# 今後必要な行だけを取り出し、リネーム
kadaiDai2 = raw_data2[["ID", "FA", "description"]]
# kadaiDai2 = raw_data2[["ID", "FA", "Title"]] # only for NoDescription
kadaiDai2['tabDai'] = 0
kadaiDai2.columns = ["ID", "FA", "Abst", "tabDai"]

# kadaiDai2 = kadaiDai2[kadaiDai2['FA'] != 'JSPS']

print("Finish")
kadaiDai2

In [ ]:
print("Start")

# 課題番号の重複を確認。課題番号でソートする。
kadaiDai2["ID"].duplicated().any()
# kadaiDai2 = kadaiDai2.set_index("ID") # IDをインデックスに設定するコード
kadaiDai2 = kadaiDai2.sort_values("ID")

# Abstが空欄の課題を削除
print("オリジナルの課題数： %5d" % len(kadaiDai2))
print("概要が空白の課題数： %5d" % len(kadaiDai2[kadaiDai2["Abst"].isna()]))
kadaiDai2 = kadaiDai2.dropna(subset=["Abst"])
print("空白を除いた課題数： %5d" % len(kadaiDai2))

# Abst中の改行コードを削除
kadaiDai2 = kadaiDai2.replace('\r', '', regex=True)
kadaiDai2 = kadaiDai2.replace('\n', '', regex=True)
kadaiDai2 = kadaiDai2.replace('　', '', regex=True) # 全角スペース

# Abstが英語のみの課題を削除
num_jpen = len(kadaiDai2)
kadaiDai2 = kadaiDai2[kadaiDai2["Abst"].str.contains(r'[ぁ-んァ-ン]')]
num_jp   = len(kadaiDai2)
print("日本語＋英語： %5d" % num_jpen)
print("英語　　　　： %5d" % (num_jpen - num_jp))
print("日本語　　　： %5d" % num_jp)

print("Finish")